In [35]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [38]:
## Creds to 'https://www.geeksforgeeks.org/house-price-prediction-using-machine-learning-in-python/ for data preprocessing

dataset = pd.read_csv('house.csv')
dataset.drop(['Id'],
             axis=1,
             inplace=True)

dataset['SalePrice'] = dataset['SalePrice'].fillna(
  dataset['SalePrice'].mean())


new_dataset = dataset.dropna()
s = (new_dataset.dtypes == 'object')
object_cols = list(s[s].index)
OH_encoder = OneHotEncoder(sparse_output=False)
OH_cols = pd.DataFrame(OH_encoder.fit_transform(new_dataset[object_cols]))
OH_cols.index = new_dataset.index
OH_cols.columns = OH_encoder.get_feature_names_out()
df_final = new_dataset.drop(object_cols, axis=1)
df_final = pd.concat([df_final, OH_cols], axis=1)

In [41]:
# Predict sales price
from sklearn.model_selection import train_test_split
X = df_final.drop('SalePrice', axis=1)  # Specify the column name for the target variable
y = df_final['SalePrice']  # Specify the column name for the target variable

# Split the data into train and test sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)


In [276]:
def relu(x):
    return np.where(x > 0, x, 0)

In [305]:
def reluderiv(x):
    return np.where(x > 0, 1, 0)
## Class For each layer
class Layer:
    def __init__(self,inputneurons,neurons,output = False) -> None:
        self.weights = np.random.rand(neurons,inputneurons) * 0.05
        self.biases = np.random.rand(neurons) * 0.05
        self.activation = None
        self.deltamatrix = None
        self.output = output
        self.outputy = None
    # Changes self.activation to the activated matrix for this layer
    def forward(self,input): 
        activation = relu(np.dot(self.weights,input))
        self.activation = activation
    def backward(self,activationbefore,layerahead = None):
        # Regression case
        # First find delta
        if self.output == True :
            if self.weights.shape[0] == 1:
                self.deltamatrix = -2 * (self.activation -  self.outputy)
        else:
            deltasoflayerbefore = layerahead.deltamatrix
            print(deltasoflayerbefore)
            weightsoflayerbefore = layerahead.weights
            maskedactivation = reluderiv(self.activation)
            self.deltamatrix = np.multiply((np.dot(weightsoflayerbefore.T,deltasoflayerbefore)),maskedactivation)
        # updates
        gradientmatrix = ((np.dot(activationbefore,self.deltamatrix.T)).T)
        self.weights = self.weights - 0.05 * (gradientmatrix)

        

In [306]:
# sfsdfwerewsdf

In [317]:

class NeuralNetwork:
    def __init__(self,hiddenlayers= [10,20]) -> None:
        self.hiddenlayers = hiddenlayers
        self.listoflayers = []
        self.type = ''
    def forwardpass(self,X):
        for i in range(len(self.listoflayers)):
            if i == 0:
                (self.listoflayers[0]).forward(X)
            else:
                (self.listoflayers[i]).forward(self.listoflayers[i-1].activation)

    def backwardpass(self,X,y):
        reverse = list(reversed(self.listoflayers))
        for i,value in enumerate(reverse):
            if i == 0:
                value.outputy = y
                value.backward(activationbefore=reverse[i + 1].activation)
            elif i == (len(reverse) - 1):
                value.backward(activationbefore=X,layerahead = reverse[i - 1])
            else:
                value.backward(activationbefore=reverse[i + 1].activation,layerahead = reverse[i - 1])

    def batchtrain(self,x,y):
        self.forwardpass(X=x)
        self.backwardpass(X = x,y=y)
    def predict(self,X,y):
        testx = X.values
        testy = y.values
        self.forwardpass(testx)
        y = testy.reshape(1,X.shape[1])
        loss = np.sum((self.listoflayers[-1].activation - y)**2)/X.shape[0]
        
        return  loss
            

    def train(self,X,y,testx,testy,batchsize = 32,epoch = 1):
        # Asserts
        assert isinstance(X,pd.DataFrame)
        for column in X.columns:
            column_type = X[column].dtype          
            assert column_type == 'float64' or column_type == 'int64'
        assert isinstance(y,pd.Series)
        sizeofinput = X.shape[1]
        for i in range(len(self.hiddenlayers)):
            if i == 0:
                self.listoflayers.append(Layer(sizeofinput,self.hiddenlayers[i]))
            else:
                self.listoflayers.append(Layer(self.hiddenlayers[i - 1],self.hiddenlayers[i]))
        ## Checks if regression, then adds an output layer with one node
        if y.dtype == 'float64':
            self.listoflayers.append(Layer(self.hiddenlayers[-1],1,output=True))
        ## Classification
        ## actual training
        x = X.values
        y = y.values
        for i in range(epoch):
            # In batches
            for start_index in np.arange(0, len(x), batchsize):
                end_index = start_index + batchsize
                x_batch = x[start_index:end_index]
                y_batch = y[start_index:end_index].reshape(1,x_batch.shape[0])
                self.batchtrain(x_batch.T,y_batch)
                print(self.predict(testx.T,testy))






In [318]:
nn = NeuralNetwork(hiddenlayers=[2,3])
nn.train(train_X,train_y,test_X,test_y)






[[361840.30712389 361839.99082383 361840.34943564 401997.69736818
  361839.39619296 361840.61078488 307797.28113802 361840.37316211
  361838.90001415 423997.48420953 361839.94434338 388997.70232141
  361840.85783437 379997.22621047 361840.39856811 206398.31851761
  361839.88961285 345997.89549494 361839.78507983 271997.79040504
  361839.75051115 361841.27329116 411998.06468785 361839.46238312
  361840.00614071 349797.62807358 361839.79012622 307997.41412969
  478997.6248114  272998.45043939 361837.37779993 309998.07237854]]
[[-6.29659388e+12 -6.29658837e+12 -6.29659462e+12 -6.99539601e+12
  -6.29657803e+12 -6.29659916e+12 -5.35615972e+12 -6.29659503e+12
  -6.29656939e+12 -7.37822711e+12 -6.29658757e+12 -6.76917553e+12
  -6.29660346e+12 -6.61255300e+12 -6.29659547e+12 -3.59165732e+12
  -6.29658661e+12 -6.02091085e+12 -6.29658479e+12 -4.73319193e+12
  -6.29658419e+12 -6.29661069e+12 -7.16941822e+12 -6.29657918e+12
  -6.29658864e+12 -6.08703221e+12 -6.29658488e+12 -5.35964235e+12
  -8.335

In [299]:
print(train_X,train_y,test_X,test_y)

      MSSubClass  LotArea  OverallCond  YearBuilt  YearRemodAdd  BsmtFinSF2   
2356          20     7441            5       2006          2006         0.0  \
1789          30    10800            5       1890          1998         0.0   
2371          20     7321            5       1999          2000         0.0   
930           20     8925            5       2007          2007         0.0   
2816          20    13050            6       1963          1963       850.0   
...          ...      ...          ...        ...           ...         ...   
1638          20    10928            6       1978          1986         0.0   
1095          20     9317            5       2006          2006         0.0   
1130          50     7804            3       1928          1950         0.0   
1294          20     8172            7       1955          1990         0.0   
860           50     7642            8       1918          1998         0.0   

      TotalBsmtSF  MSZoning_C (all)  MSZoning_FV  M

In [300]:
train_X

,MSSubClass,LotArea,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF2,TotalBsmtSF,MSZoning_C (all),MSZoning_FV,MSZoning_RH,...,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing
2356,20,7441,5,2006,2006,0.0,1461.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1789,30,10800,5,1890,1998,0.0,630.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2371,20,7321,5,1999,2000,0.0,1339.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
930,20,8925,5,2007,2007,0.0,1466.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2816,20,13050,6,1963,1963,850.0,1000.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,20,10928,6,1978,1986,0.0,1427.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1095,20,9317,5,2006,2006,0.0,1314.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1130,50,7804,3,1928,1950,0.0,1122.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1294,20,8172,7,1955,1990,0.0,864.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [301]:
test_X

,MSSubClass,LotArea,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF2,TotalBsmtSF,MSZoning_C (all),MSZoning_FV,MSZoning_RH,...,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing
178,20,17423,5,2008,2009,0.0,2216.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
678,20,11844,5,2008,2008,0.0,2046.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2878,50,10164,5,1939,1950,0.0,992.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2691,20,7733,5,2005,2005,0.0,1142.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1898,30,8574,8,1916,2000,0.0,816.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,20,8250,5,1968,1968,492.0,952.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
542,20,10140,5,1998,1999,1127.0,1650.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2319,20,7697,6,2007,2007,0.0,1246.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1644,20,9373,7,1975,1975,168.0,1621.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [302]:
test_y

178     501837.00000
678     372500.00000
2878    180921.19589
2691    180921.19589
1898    180921.19589
            ...     
1221    134000.00000
542     213250.00000
2319    180921.19589
1644    180921.19589
1591    180921.19589
Name: SalePrice, Length: 583, dtype: float64